# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from pprint import pprint
# Configure gmaps
gmaps.configure(api_key=g_key)

#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#heatmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
#import cities csv 
#file_path=os.path.join('../', 'output_data','weather_data.csv')

In [42]:
file_path= os.path.join('../', 'WeatherPy/','weather_data_one.csv')
cities_df=pd.read_csv(file_path) 
cities_df=pd.read_csv(file_path, index_col=0)
cities_df
#Note: index_col=False can be used to force pandas to not use the first column as the index, e.g. 
#when you have a malformed file with delimiters at the end of each line.

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,286.15,62,75,7.20,AR,1612838864
1,Coos Bay,43.3665,-124.2179,281.15,71,40,4.12,US,1612838750
2,Pochutla,15.7432,-96.4661,298.71,73,0,1.44,MX,1612838996
3,Maningrida,-12.0667,134.2667,304.43,63,99,4.42,AU,1612838996
4,Santa Rosa,14.3122,121.1114,299.82,83,90,1.54,PH,1612838996
...,...,...,...,...,...,...,...,...,...
550,Jishou,28.3167,109.7167,284.84,67,100,1.29,CN,1612839104
551,Luzhou,28.8724,105.4405,289.70,54,100,1.06,CN,1612839105
552,Lorengau,-2.0226,147.2712,302.17,77,1,3.31,PG,1612839105
553,Bedelē,8.4560,36.3530,284.27,50,23,1.00,ET,1612839105


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [43]:
#Use Lat and Lon for locations
# weights equal to humidity
locations= cities_df[["Latitude", "Longitude"]]
weights=cities_df['Humidity'] #where is humidity most intense 

In [44]:
fig = gmaps.figure()

In [45]:
cities_df.columns

Index(['City', 'Latitude', 'Longitude', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [49]:
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=weights, 
                               dissipating=False, max_intensity=100,
                               point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Fill Nan values and convert to float
# rating= airport_df["Airport Rating"].astype(float)

In [7]:
#* Narrow down the DataFrame to find your ideal weather condition. For example:
#  * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.

# Create a data frame using a list within cities_df. #A big filter
# cloudiness is equal to 15. 
# Wind speed is les than 10
# Max Temp is greater than 60 but less than 80 (2 calls)

result_df=cities_df[(cities_df['Cloudiness']<5)&(cities_df['Wind Speed']<10)&(cities_df['Max Temp']>340)&(cities_df['Max Temp']<350)]
result_df.head()
#print(ideal_weather_df)

# WHY WONT THIS SHOW RESULT_DF???

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [9]:
# rank order cloud > wind> max temp,

cloud_filter=cities_df.loc[:, 'Cloudiness']<7
cloud_ideal=cities_df.loc[cloud_filter]
#cloud_ideal.head()
#cloudiness is number one preference

wind_filter=cloud_ideal.loc[:, 'Wind Speed']<25
wind_ideal=cloud_ideal.loc[wind_filter]
#wind_ideal

temp_filter=(wind_ideal['Max Temp']>294)&(wind_ideal['Max Temp']<300)  #(cities_df['Max Temp']>340)&(cities_df['Max Temp']<350)
temp_ideal=wind_ideal[temp_filter]
temp_ideal

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,Wukari,7.8500,9.7833,296.92,20,0,1.61,NG,1612050364
67,San Ignacio,17.1561,-89.0714,298.15,73,0,2.06,BZ,1612050367
97,Touros,-5.1989,-35.4608,299.15,83,0,2.57,BR,1612050208
105,Beloha,-25.1667,45.0500,297.01,80,2,1.62,MG,1612050245
109,Umm Hājar,13.2954,19.6966,294.36,22,0,3.74,TD,1612050374
114,Cap Malheureux,-19.9842,57.6142,297.59,87,4,0.89,MU,1612050177
119,Saint-Philippe,-21.3585,55.7679,299.15,65,0,2.06,RE,1612050009
131,Caravelas,-17.7125,-39.2481,297.59,86,3,2.69,BR,1612050189
152,Am Timan,11.0297,20.2827,295.18,21,2,3.44,TD,1612050065
164,Pisco,-13.7000,-76.2167,296.15,68,0,8.23,PE,1612050149


In [10]:
#cloud_ideal

In [11]:
#wind_ideal

In [12]:
#temp_ideal

In [13]:
#wind_speed_filter=cloud_ideal.loc[:, 'Wind Speed']<40
#wind_speed=wind_speed.loc[wind_speed_filter, :]
#wind_speed.head()

In [27]:
temp_ideal["Hotel Name"]= " "
hotel_df=pd.DataFrame(temp_ideal)
hotel_df.head(10)

<ipython-input-27-13989a615fe8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_ideal["Hotel Name"]= " "


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,Wukari,7.8500,9.7833,296.92,20,0,1.61,NG,1612050364,
67,San Ignacio,17.1561,-89.0714,298.15,73,0,2.06,BZ,1612050367,
97,Touros,-5.1989,-35.4608,299.15,83,0,2.57,BR,1612050208,
105,Beloha,-25.1667,45.0500,297.01,80,2,1.62,MG,1612050245,
109,Umm Hājar,13.2954,19.6966,294.36,22,0,3.74,TD,1612050374,
114,Cap Malheureux,-19.9842,57.6142,297.59,87,4,0.89,MU,1612050177,
119,Saint-Philippe,-21.3585,55.7679,299.15,65,0,2.06,RE,1612050009,
131,Caravelas,-17.7125,-39.2481,297.59,86,3,2.69,BR,1612050189,
152,Am Timan,11.0297,20.2827,295.18,21,2,3.44,TD,1612050065,
164,Pisco,-13.7000,-76.2167,296.15,68,0,8.23,PE,1612050149,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
#geocordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set the parms used in teh API call
lat_long=""
distance=5000
target_type='lodging'

# how do you get the whole list?
#target_coordinates= '63.7506, -68.5145'
#target_radius= 5000
#target_type='hotel' #can search and type be the same? they are different H vs h but not sure?
# set up a parameters dictionary

query_params= {
    "location": lat_long,
    "radius" : distance,
    "type" : target_type,
    "key" : g_key
}

#for index, each_row in hotel_df.head(1).iterrows():
for index, each_row in hotel_df.iterrows():
    try:
        query_params['location']=f"{each_row['Latitude']},{each_row['Longitude']}"
        
        # Compile Query URL and make API request
        response=requests.get(base_url,params=query_params)
        data=response.json() #giving response 200
        #zero index and the name key
        hotel_name=data['results'][0]['name']
        # Class acitivty maybe airport 
        #cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, 'Hotel Name']=hotel_name
        #pprint(data)
    except:
        print(f"Unable to locate a hotel in {each_row['City']}")

Unable to locate a hotel in Beloha
Unable to locate a hotel in Umm Hājar
Unable to locate a hotel in Yaan
Unable to locate a hotel in Uvalde
Unable to locate a hotel in Doka


In [18]:
#response.url

In [51]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,Wukari,7.8500,9.7833,296.92,20,0,1.61,NG,1612050364,Elimtop
67,San Ignacio,17.1561,-89.0714,298.15,73,0,2.06,BZ,1612050367,Rainforest Haven Inn
97,Touros,-5.1989,-35.4608,299.15,83,0,2.57,BR,1612050208,INN NEW HORIZON
105,Beloha,-25.1667,45.0500,297.01,80,2,1.62,MG,1612050245,
109,Umm Hājar,13.2954,19.6966,294.36,22,0,3.74,TD,1612050374,
114,Cap Malheureux,-19.9842,57.6142,297.59,87,4,0.89,MU,1612050177,Pereybere Hotel & Spa
119,Saint-Philippe,-21.3585,55.7679,299.15,65,0,2.06,RE,1612050009,"Chambres d'hôte ""La Trinité"""
131,Caravelas,-17.7125,-39.2481,297.59,86,3,2.69,BR,1612050189,Pousada dos Navegantes
152,Am Timan,11.0297,20.2827,295.18,21,2,3.44,TD,1612050065,Palais du chef Canton Salamt
164,Pisco,-13.7000,-76.2167,296.15,68,0,8.23,PE,1612050149,Hotel San Isidro Oficial


In [52]:
#len(hotel_df) 
# gives you the full hotel data frame fitler down to ideal

In [53]:
# Drop any row for NA
hotel_1_df=hotel_df.dropna(axis=0, how='any') #axis=0 means drop a row. axis=1 means drop a column
#len(hotel_1_df) # gives you the full hotel data frame without NAN. 


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_1_df.iterrows()]
#hotel_info
locations = hotel_1_df[["Latitude", "Longitude"]]

In [55]:
# Add marker layer ontop of heat map
fig=gmaps.figure()

markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:


# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_1_df.iterrows()]
# locations = hotel_1_df[["Latitude", "Longitude"]]

# # Add marker layer ontop of heat map
# fig=gmaps.figure()

# markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
# fig.add_layer(heat_layer)
# fig.add_layer(markers)


# # Display figure
# fig

#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#heatmaps

Figure(layout=FigureLayout(height='420px'))